In [14]:
import pandas as pd
import numpy as np

aadt = pd.read_csv("C:/Users/DuminduS/Desktop/UWE/CSCT Masters Project/datasets/bristol_traffic_average_day.csv")
aadt.columns = [c.strip().lower().replace(" ", "_") for c in aadt.columns]

# Keep main columns
aadt = aadt[["count_point_id", "latitude", "longitude", "all_motor_vehicles"]].drop_duplicates()

aadt.head()


,count_point_id,latitude,longitude,all_motor_vehicles
0,6401,51.436429,-2.605614,6088
1,6401,51.436429,-2.605614,5969
2,6401,51.436429,-2.605614,5952
3,6401,51.436429,-2.605614,5831
4,6401,51.436429,-2.605614,6309


In [16]:
stations = pd.read_csv(
    "C:/Users/DuminduS/Desktop/UWE/CSCT Masters Project/datasets/Final_dataset.csv"
)[["station_id", "latitude", "longitude"]].drop_duplicates()


In [18]:
# Haversine distance in km
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c
# For each station, find nearest sensor
station_ids = []
nearest_traffic = []

sensor_lats = aadt["latitude"].values
sensor_lons = aadt["longitude"].values
sensor_counts = aadt["all_motor_vehicles"].values

for _, st in stations.iterrows():
    lat_s = st["latitude"]
    lon_s = st["longitude"]
    dists = haversine(lat_s, lon_s, sensor_lats, sensor_lons) # distances to all sensors
    idx_min = np.argmin(dists)

    station_ids.append(st["station_id"])
    nearest_traffic.append(sensor_counts[idx_min])

station_traffic = pd.DataFrame({
    "station_id": station_ids,
    "avg_daily_traffic_nearby": nearest_traffic
})

station_traffic.head()


,station_id,avg_daily_traffic_nearby
0,BRI_0000,8465
1,BRI_0001,29273
2,BRI_0002,8465
3,BRI_0003,358
4,BRI_0004,3936


In [20]:
daily = pd.read_csv(
    "C:/Users/DuminduS/Desktop/UWE/CSCT Masters Project/datasets/Final_dataset.csv"
)

# Parse dates
daily["date"] = pd.to_datetime(daily["date"], format="mixed", dayfirst=True, errors="coerce")

# Drop rows where date cannot parsed
daily = daily.dropna(subset=["date"])

# Merge static traffic feature
daily = daily.merge(station_traffic, on="station_id", how="left")

# Save output
daily.to_csv(
    "C:/Users/DuminduS/Desktop/UWE/CSCT Masters Project/datasets/Final02.csv",
    index=False
)

daily.head()



,station_id,date,sessions,energy_kwh,mean_charge_duration_min,latitude,longitude,n_connectors,city_daily_traffic,temperature_2m_max,temperature_2m_min,temperature_2m_mean,precipitation_sum,rain_sum,snowfall_sum,windspeed_10m_max,windspeed_10m_mean,avg_daily_traffic_nearby
0,BRI_0000,2024-01-01,22,602.991,133.911205,51.542927,-2.568751,6,NaN,12.0,7.3,8.7,16.6,16.6,0.0,43.0,25.5,8465
1,BRI_0000,2024-01-02,44,984.970,136.226415,51.542927,-2.568751,6,NaN,13.4,9.1,10.7,8.7,8.7,0.0,53.7,29.7,8465
2,BRI_0000,2024-01-03,32,582.430,136.061438,51.542927,-2.568751,6,NaN,9.7,7.8,8.7,2.2,2.2,0.0,27.9,23.8,8465
3,BRI_0000,2024-01-04,32,753.940,161.179459,51.542927,-2.568751,6,NaN,8.3,5.4,6.8,8.4,8.4,0.0,28.5,16.8,8465
4,BRI_0000,2024-01-05,40,797.936,114.216709,51.542927,-2.568751,6,NaN,7.5,3.8,5.3,0.0,0.0,0.0,14.5,9.9,8465
